In [ ]:
import filterServices
import H5toDataframe as readH5
import outlierServices
import clusterServices
import linRegServices
import conversionServices as convServices

# H5 Dateien einlesen
all_dataframes = readH5.convertAllH5Files()

# Easteregg rows entfernen
all_dataframes = filterServices.filterEasterEggs(all_dataframes)

# Übrige strings in floats konvertieren
all_dataframes = convServices.convert_all_strings_to_floats(all_dataframes)

# Timestamp outlier entfernen
all_dataframes = outlierServices.removeOutliersTurkeysMethod_withMean(all_dataframes, 'timestamp')

# Berechnen der fehlenden Velocity Werte
all_dataframes = convServices.calculate_velocity_conditionally(all_dataframes)

# wall_thickness outlier entfernen
all_dataframes = outlierServices.removeOutliersTurkeysMethod_withForwardAndBackFill(all_dataframes, 'wall_thickness')

# NaN Werte rausfiltern
all_dataframes = filterServices.replace_nan_with_avg_of_neighbors(all_dataframes, 'wall_thickness')
all_dataframes = filterServices.replace_nan_with_avg_of_neighbors(all_dataframes, 'magnetization')

# wall_thickness cluster Bestimmen, durch Mittelwert ersetzen und letzte Outlier rausfiltern
all_dataframes = clusterServices.clusterWallThicknessAndCalculateMean(all_dataframes)

# Abnutzung der magnetization mittels linearer Regression rausrechnen
all_dataframes = linRegServices.clean_magnetization_with_linReg(all_dataframes)


In [ ]:
for name, (df, attrs) in all_dataframes.items():
    print(df.head())

# Upload to Influx

In [ ]:
import influxServices

influxServices.upload_dict_with_frames(all_dataframes, 'bucket11')